아래와 같이 텍스트 파일을 읽어 옵니다. mypath라는 변수에 입력된 폴더의 파일들을 모두 가져오는 명령문이다. 

In [2]:
from os import listdir
from os.path import isfile, join

mypath = './example_En_docs/'

In [3]:
listdir(mypath)

['doc3_1.txt',
 'doc1_2.txt',
 'doc3_2.txt',
 'doc1_1.txt',
 'doc3_3.txt',
 'doc3_4.txt',
 'doc2_2.txt',
 'doc2_3.txt',
 'doc0_1.txt',
 'doc0_3.txt',
 'doc2_1.txt',
 'doc0_2.txt',
 'doc4_2.txt',
 'doc4_3.txt',
 'doc4_1.txt']

In [6]:
onlyfiles = []
for f in listdir(mypath):
    if isfile(join(mypath, f)):
        onlyfiles.append(f)
        
# 위 코드는 아래와 동일합니다.
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [7]:
total_docs = []
for file in onlyfiles:
    file_path = mypath+file
    with open(file_path, 'r', encoding='utf8') as f:
        content = f.read()
    total_docs.append(content)

In [10]:
import preprocessing # import a python file for English text preprocessing

In [11]:
stopwords = ['be', 'today', 'yesterday', 'new', 'york', 'time'] # 불용어 사전 생성하기

하나의 문서에 대해서 하나의 벡터를 형성시킨다. 이를 각 문서마다 적용시키면 최종적으로 하나의 matirx를 얻을 수 있다.

In [13]:
docs = [preprocessing.En_processing(doc, stopwords) for doc in total_docs]
# docs의 각 원소는 문서 하나에 대해서 (문서 내용, [명사단어])로 구성
# 문서 내용은 문장들로 구성

In [17]:
docs[0]

('Growth in Chinas manufacturing sector sputtered in September as both external and domestic demand weakened two surveys showed on Sunday raising the pressure on policymakers as U.S. tariffs appear to be inflicting a heavier toll on the Chinese economy.\n\nA private survey showed growth in the factory sector stalled after 15 months of expansion with export orders falling the fastest in over two years while an official survey confirmed a further manufacturing weakening.\nTaken together the business activity gauges  the first major readings on Chinas economy for September  confirm consensus views that the worlds secondlargest economy is continuing to cool which is likely to prompt Chinese policymakers to roll out more growthsupport measures in coming months.\n\nWe should make policy preparations as the external pressure on the economy is rising and it will increase further next year said Tang Jianwei senior economist at Bank of Communications in Shanghai.\n\nSome cushion for the slowing 

In [18]:
DOCS_NUM = len(docs) 

In [19]:
DOCS_NUM

15

모든 문서에서 사용된 모든 명사를 저장한 것이, total_docs_nouns이고, 각 문서 안에서 명사를 모두 저장한 것이 docs_nouns이다.

In [20]:
total_docs_nouns = [] # corpus에 사용된 모든 단어 저장
docs_nouns = [] # 각 문서를 단어로 표현한 것들의 리스트
for doc, words in docs:
    total_docs_nouns.extend(words)
    docs_nouns.append(words)

In [23]:
len(docs_nouns)

15

In [24]:
len(total_docs_nouns)

3203

In [25]:
len(set(total_docs_nouns))

1168

In [26]:
from sklearn.feature_extraction.text import CountVectorizer # frequency based DTM
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf based DTM

In [27]:
def tf_extractor(corpus): 
    # returns a frequency-based DTM
    vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1))
    # min_df: minimum document frequency threshold
         # if min_df = 1 -> 최소 적어도 하나의 문서에서 사용된 단어들은 모두 포함
    # ngram_range=(1,1) -> unigram 만 포함
    # ngram_range=(1,3) -> unigram, bi-gram, tri-gram 까지 고려
    features = vectorizer.fit_transform(corpus) # transform texts to a frequency matrix
    return vectorizer, features  

In [28]:
def tfidf_extractor(corpus):
    # returns a tf-idf based DTM
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [71]:
print(docs_nouns[0])

['growth', 'manufacturing', 'sector', 'september', 'demand', 'survey', 'sunday', 'pressure', 'policymakers', 'toll', 'economy', 'survey', 'growth', 'factory', 'sector', 'month', 'expansion', 'export', 'order', 'year', 'survey', 'manufacturing', 'business', 'activity', 'reading', 'china', 'economy', 'consensus', 'view', 'world', 'economy', 'policymakers', 'measure', 'month', 'policy', 'preparation', 'pressure', 'economy', 'year', 'tang', 'jianwei', 'economist', 'bank', 'communication', 'cushion', 'economy', 'service', 'half', 'economy', 'purchasing', 'manager', 'index', 'pmi', 'bureau', 'statistic', 'sunday', 'service', 'faster', 'rate', 'september', 'index', 'low', 'september', 'august', 'reuters', 'forecast', 'index', 'mark', 'expansion', 'contraction', 'month', 'caixinmarkit', 'purchasing', 'manager', 'index', 'pmi', 'economist', 'reuters', 'average', 'survey', 'september', 'china', 'factory', 'business', 'activity', 'export', 'order', 'official', 'data', 'number', 'company', 'poll',

아래의 과정을 거치게 되면, 최종적으로 15개의 원소가 생긴다.

In [29]:
documents_filtered = []
for doc in docs_nouns:
    document_filtered =''
    for word in doc:
        document_filtered = document_filtered+' '+word
    documents_filtered.append(document_filtered) # list of docs, doc 은 단어들로 구성이 된 string data
# 불필요한 단어들을 제거하고 난후 DTM로 변환하기 위해 다시 list of strings의 형태로 변환

In [31]:
documents_filtered

[' growth manufacturing sector september demand survey sunday pressure policymakers toll economy survey growth factory sector month expansion export order year survey manufacturing business activity reading china economy consensus view world economy policymakers measure month policy preparation pressure economy year tang jianwei economist bank communication cushion economy service half economy purchasing manager index pmi bureau statistic sunday service faster rate september index low september august reuters forecast index mark expansion contraction month caixinmarkit purchasing manager index pmi economist reuters average survey september china factory business activity export order official data number company poll firm job creation official jo loss trade risk caixin survey export order indicator activity pace company order friction tariff survey export order contracting month expansion manufacturing sector september export performance demand impact company production zhengsheng zhon

각각의 cell이 tfidf에 대한 값을 포함하고 있는 행렬을 생성한다.

In [32]:
vectorizer_tfidf, doc_term_mat_tfidf = tfidf_extractor(documents_filtered) # 문서들을 TFIDF matrix로 변환

In [33]:
vectorizer_tf, doc_term_mat_tf = tf_extractor(documents_filtered) # 문서들을 TF matrix로 변환

In [34]:
print(doc_term_mat_tf)

  (0, 479)	6
  (0, 630)	3
  (0, 923)	4
  (0, 934)	6
  (0, 305)	4
  (0, 1026)	6
  (0, 1021)	2
  (0, 789)	3
  (0, 764)	3
  (0, 1068)	1
  (0, 344)	9
  (0, 397)	2
  (0, 668)	6
  (0, 388)	3
  (0, 392)	9
  (0, 712)	6
  (0, 1160)	4
  (0, 146)	3
  (0, 8)	3
  (0, 842)	1
  (0, 191)	8
  (0, 239)	1
  (0, 1117)	1
  (0, 1155)	1
  (0, 644)	3
  :	:
  (14, 604)	1
  (14, 577)	1
  (14, 300)	1
  (14, 837)	1
  (14, 408)	1
  (14, 290)	1
  (14, 574)	1
  (14, 316)	1
  (14, 813)	1
  (14, 831)	3
  (14, 260)	1
  (14, 986)	1
  (14, 208)	1
  (14, 719)	1
  (14, 1052)	1
  (14, 976)	2
  (14, 353)	1
  (14, 94)	1
  (14, 714)	1
  (14, 79)	1
  (14, 121)	1
  (14, 303)	1
  (14, 20)	1
  (14, 183)	1
  (14, 368)	1


dense array는 0을 포함한 array이다.

In [36]:
import numpy as np

In [37]:
DTM_TF = np.array(doc_term_mat_tf.todense()) #dense array (즉, 0을 포함한 array)로 변환후 다시 numpy array로 변환

In [38]:
DTM_TF

array([[0, 0, 0, ..., 0, 1, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

문서의 수와 단어의 수를 알 수 있다.

In [80]:
DTM_TF.shape

(15, 1165)

In [54]:
DTM_TFIDF = np.array(doc_term_mat_tfidf.todense())

In [55]:
DTM_TFIDF

array([[0.        , 0.        , 0.        , ..., 0.        , 0.04116293,
        0.04116293],
       [0.        , 0.02776663, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04711425, 0.0409108 , 0.04711425, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

거리가 짧을수록 더 유사한 벡터이다. 첫 번째와 두번째, 첫 번재와 네 번째, 첫 번째와 6번째 문서 간의 유클리디안 거리이며 이는 TF 값을 기준으로 계산한다.

In [56]:
d1d2_tf = np.linalg.norm(DTM_TF[0]-DTM_TF[1])
d1d4_tf = np.linalg.norm(DTM_TF[0]-DTM_TF[3])
d1d6_tf = np.linalg.norm(DTM_TF[0]-DTM_TF[5])
print(d1d2_tf, d1d4_tf, d1d6_tf)

50.635955604688654 36.013886210738214 19.493588689617926


절대적인 값 자체는 중요하지 않다. 중요한 것은 상대적인 값의 차이이다. 아래의 결과를 보면 우리는 TFIDF값이 TF보다 더 타당성 있는 계산을 한다는 점을 유추해볼 수 있다.

In [57]:
d1d2_tfidf = np.linalg.norm(DTM_TFIDF[0]-DTM_TFIDF[1])
d1d4_tfidf = np.linalg.norm(DTM_TFIDF[0]-DTM_TFIDF[3])
d1d6_tfidf = np.linalg.norm(DTM_TFIDF[0]-DTM_TFIDF[5])
print(d1d2_tfidf, d1d4_tfidf, d1d6_tfidf)

1.3819550784841121 1.3923266882534269 0.784074389523561


아래는 코사인 유사도 계산이다. 보면 잘 알 수 있는데, 코사인 유사도가 훨씬 더 직관적으로 이해하기 쉽게 유사도를 알려준다.

In [58]:
d1d2_cos_tf = np.dot(DTM_TF[0],DTM_TF[1])/(np.linalg.norm(DTM_TF[0])*np.linalg.norm(DTM_TF[1]))
d1d4_cos_tf = np.dot(DTM_TF[0],DTM_TF[3])/(np.linalg.norm(DTM_TF[0])*np.linalg.norm(DTM_TF[3]))
d1d6_cos_tf = np.dot(DTM_TF[0],DTM_TF[5])/(np.linalg.norm(DTM_TF[0])*np.linalg.norm(DTM_TF[5]))
print(d1d2_cos_tf, d1d4_cos_tf, d1d6_cos_tf)

0.0699569668707187 0.04300167238072299 0.7624928516630234


In [65]:
d1d2_cos_tfidf = np.dot(DTM_TFIDF[0],DTM_TFIDF[1])/(np.linalg.norm(DTM_TFIDF[0])*np.linalg.norm(DTM_TFIDF[1]))
d1d4_cos_tfidf = np.dot(DTM_TFIDF[0],DTM_TFIDF[3])/(np.linalg.norm(DTM_TFIDF[0])*np.linalg.norm(DTM_TFIDF[3]))
d1d6_cos_tfidf = np.dot(DTM_TFIDF[0],DTM_TFIDF[5])/(np.linalg.norm(DTM_TFIDF[0])*np.linalg.norm(DTM_TFIDF[5]))
print(d1d2_cos_tfidf, d1d4_cos_tfidf, d1d6_cos_tfidf)

0.045100080525985194 0.030713196588621965 0.6926136758466276


In [60]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity # cosine similarity 계산하기

tfidf 방식을 이용한 문서들 간의 유사도 계산

In [67]:
d1d2_cos_idf = cosine_similarity(DTM_TFIDF[0].reshape(1,-1), DTM_TFIDF[1].reshape(1,-1))
d1d4_cos_idf = cosine_similarity(DTM_TFIDF[0].reshape(1,-1), DTM_TFIDF[3].reshape(1,-1))
d1d5_cos_idf = cosine_similarity(DTM_TFIDF[0].reshape(1,-1), DTM_TFIDF[5].reshape(1,-1))
d1d7_cos_idf = cosine_similarity(DTM_TFIDF[0].reshape(1,-1), DTM_TFIDF[6].reshape(1,-1))
d1d10_cos_idf = cosine_similarity(DTM_TFIDF[0].reshape(1,-1), DTM_TFIDF[9].reshape(1,-1))
print(d1d2_cos_idf, d1d4_cos_idf, d1d5_cos_idf, d1d7_cos_idf, d1d10_cos_idf)

[[0.04510008]] [[0.0307132]] [[0.69261368]] [[0.08835517]] [[0.02950616]]


이제 K-Means를 계산하자. 이때, 먼저 KMeans와 동일한 카피본의 변수를 만들어야 한다. 중요한 파라미터는 'n_clusters'인데, 이를 통해서 우리는 사전에 몇 개의 군집으로 나눌지 지정할 수 있다. 

In [68]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, max_iter=10, random_state=0)
clusters = kmeans.fit_predict(DTM_TFIDF) # KMeans 알고리즘으로 학습

아래의 숫자들은 군집의 ID이다. 즉 각각의 문서들이 어떠한 ID를 가지고 있는지에 따라서 군집이 나뉘어진 것이다.

In [71]:
clusters

array([2, 3, 2, 3, 2, 2, 1, 1, 3, 3, 1, 3, 0, 0, 0], dtype=int32)

In [72]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, max_iter=10, random_state=0)
clusters = kmeans.fit_predict(DTM_TFIDF) # KMeans 알고리즘으로 학습

이번에는 K의 값을 5로 지정해서 한 것이다. 이러한 결과를 통해서 우리가 직접 주제를 알 수는 없다.

In [73]:
clusters

array([2, 3, 2, 3, 2, 2, 1, 1, 4, 4, 1, 4, 0, 0, 0], dtype=int32)

현실적으로는, 연구할 때 K의 값을 크게 지정한다. 그리고 사람이 살펴보고, 만약 직관과 다른 결과가 들어있는 분석이라면 배제하는 방식으로 최종적으로 직관 혹은 이론과 가장 유사한 결과물들을 가지고 비교하는 수밖에 없다.